In [1]:
import cv2
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
from astropy.io import fits
from skimage.transform import hough_line, hough_line_peaks
from skimage.transform import probabilistic_hough_line
from skimage.feature import canny
from skimage.draw import line
from skimage import data
from timeit import default_timer as timer
from scipy.ndimage.filters import gaussian_filter
import numpy.ma as ma
import glob
import os
import pandas as pd
from scipy import ndimage
from skimage.morphology import square, dilation 
import math
%matplotlib widget

In [2]:
#Directory where smaps of splitted images and full image is stored
dataset = 'short'
image_folder_smaps = '/lustre/fs23/group/icecube/azenin/last/images/last_streaks/last_' + dataset + '_part/'
image_folder_image = '/lustre/fs23/group/icecube/azenin/last/images/last_streaks/last_' + dataset + '_new.fit'
full_image_file = fits.open(image_folder_image)
full_image = full_image_file[0].data
#create empty mask for full image
mask = np.zeros((full_image.shape[0],full_image.shape[1]))
xmax, ymax = mask.shape
#properties of subimages and internal stuff
x_parts = 10
y_parts = 5
smap_file = fits.open(image_folder_smaps + 'last_' + dataset + '_x0y0_smap.fit')
smap = smap_file[0].data
cutoff_smaps = np.zeros((x_parts, y_parts, smap.shape[0], smap.shape[1]))
lines_sample = []
global_lines = []

In [3]:
#IMPORTANT: WE NEED PSF (in pixel size), 
psf=1.5

In [4]:
#Load smaps from splitted input image, calculate cutoff
line_array = np.zeros((x_parts, y_parts, 2))
for i in range(x_parts):
    for j in range(y_parts):
        filename = image_folder_smaps + 'last_' + dataset + '_x%d' % i + 'y%d' % j + '_smap.fit' 
        with fits.open(filename) as hdul:
            image=hdul[0].data
            image[np.isnan(image)] = 0
            histogram = image.flatten()
            """
            plt.hist(histogram, 100)
            plt.xlim(0,10)
            plt.yscale('log')
            """
            try:
                histogram=histogram[histogram < 0]
                quantile = np.quantile(histogram, 0.32)
            except:
                quantile = 0
            cutoff = abs(quantile)
            image[image < cutoff] = 0
            cutoff_smaps[i][j] = image
            """
            plt.imshow(image, cmap='gray')
            plt.colorbar()
            plt.show()
            """

In [18]:
global_lines = []
#Do Hough
#alignment checker
#get a and b:
def getab(streak, transpose = 0):
    x0 = streak[0][1]
    y0 = streak[0][0]
    x1 = streak[1][1]
    y1 = streak[1][0]
    if(transpose):
        x0 = streak[0][0]
        y0 = streak[0][1]
        x1 = streak[1][0]
        y1 = streak[1][1]
    if x0 == x1:
        a = np.nan
        b = np.nan
        return a, b, x0
    else:
        a = (y1 - y0)/(x1 - x0)
        b = y0 - x0 * a
    return a, b, -1

def getdist(streak):
    xd=streak[0][0]-streak[1][0]
    yd=streak[0][1]-streak[1][1]
    return np.sqrt(np.power(xd, 2) + np.power(yd, 2))
    
    
for i in range(cutoff_smaps.shape[0]):
    for j in range(cutoff_smaps.shape[1]):
        start2 = timer()
        #edges = canny(cutoff_smaps[i][j], 5*psf, 0.8)
        lines = list(probabilistic_hough_line(cutoff_smaps[i][j], threshold=400, line_length=int(50*psf), line_gap=0))
        for k in range(len(lines)):
            #check alignment
            flag = 0
            a_curr, b_curr, x_curr = getab(lines[k])
            a_curr_tp, b_curr_tp, x_curr_tp = getab(lines[k], transpose = 1)
            #check length
            dist = getdist(lines[k])
            if dist < 30*psf:
#                 print('line is too short')
                continue
            lines[k] = list(lines[k])
            lines[k][0] = list(lines[k][0])
            lines[k][1] = list(lines[k][1])
            lines[k][0][0] += j*smap.shape[1]
            lines[k][1][0] += j*smap.shape[1]
            lines[k][0][1] += i*smap.shape[0]
            lines[k][1][1] += i*smap.shape[0]
#             print("modified: %d, %d; i = %d, j = %d" % (lines[k][0][0], lines[k][0][1], i, j))
#             if(lines[k][0][0] < (i+1) * smap.shape[0] and lines[k][0][1] < (j+1) * smap.shape[1] and lines[k][1][0] < (i+1) * smap.shape[0] and lines[k][1][1] < (j+1) * smap.shape[1]):
            global_lines.append(lines[k])
print(len(global_lines))
#output is list of arrays, where start and end-points of found lines are listed
#not a list of tuples or lists?

234


In [19]:
###### sort streaks by length
# create a pandas dataframe
streaks = np.zeros((len(global_lines), 5))
for i in range(streaks.shape[0]):
    streaks[i][0] = global_lines[i][0][0]
    streaks[i][1] = global_lines[i][0][1]
    streaks[i][2] = global_lines[i][1][0]
    streaks[i][3] = global_lines[i][1][1]
    streaks[i][4] = getdist(global_lines[i])
df = pd.DataFrame(streaks, columns = ["x0", "y0", "x1", "y1", "length"])
df = df.assign(group = -1)
df["x0"] = df["x0"].astype(np.int32)
df["y0"] = df["y0"].astype(np.int32)
df["x1"] = df["x1"].astype(np.int32)
df["y1"] = df["y1"].astype(np.int32)
df = df.sort_values("length", ascending = [False]).reset_index(drop=True)
df
# df.dtypes

,x0,y0,x1,y1,length,group
0,1951,1913,1485,957,1063.528091,-1
1,1948,1913,1636,1273,712.000000,-1
2,1950,1913,1670,1340,637.753087,-1
3,2229,2464,1960,1914,612.258932,-1
4,2230,2464,1962,1914,611.820235,-1
...,...,...,...,...,...,...
229,1429,6930,1354,6907,78.447435,-1
230,1428,6931,1353,6908,78.447435,-1
231,3208,2905,3285,2901,77.103826,-1
232,1899,7062,1824,7045,76.902536,-1


In [20]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(full_image, cmap=cm.gray, norm=LogNorm())
ax[0].set_title('Input image')

ax[1].imshow(full_image * 0)
for i in range(df.shape[0]):
    p0, p1 = [df["x0"][i], df["y0"][i]], [df["x1"][i], df["y1"][i]]
    ax[1].plot((float(p0[0]), float(p1[0])), (float(p0[1]), float(p1[1])))
    #ax[1].set_xlim((0,full_image.shape[1]))
#ax[1].set_ylim((full_image.shape[0], 0))
ax[1].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()
#Here we need the code to get the global coordinates for the found lines in lines_sample
# line_array = np.zeros((x_parts, y_parts, 2))
# for i in range(x_parts):
#     for j in range(y_parts):
#         line_array[i][j][0] = lines[i*y_parts+j][0]
#         line_array[i][j][1] = lines[i*y_parts+j][1]
#Output should be an list of all global line coordinates (global_lines)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
mask = np.zeros((df.shape[0]+1,full_image.shape[0],full_image.shape[1]), dtype = np.bool_)
#Get with start and end point of lines from hough all touching array cells
def drawLine(image, x0, y0, x1, y1):
    xl, yl = line(x0, y0, x1, y1)
    image[xl, yl] = 1

    
#Here we need global_lines from the missing part before
#Place lines in the empty mask
for i in range(df.shape[0]):
    drawLine(mask[0], df["y0"][i], df["x0"][i], df["y1"][i], df["x1"][i])
    drawLine(mask[i+1], df["y0"][i], df["x0"][i], df["y1"][i], df["x1"][i])
    
#Fold mask with gaussian to get line width (PSF-based)
%matplotlib widget
# mask[0] = dilation(mask[0], disk(10))
# mask[mask > 0.] = 1
#Get masked image
masked_image = ma.masked_where(mask[0] > 0, full_image)
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(masked_image, norm=LogNorm())
ax[0].set_title("masked image")
# plt.show()
ax[1].imshow(full_image, norm=LogNorm())
# plt.colorbar()
ax[1].set_title("input image")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# mask_corr
im_w = full_image.shape[0]
im_h = full_image.shape[1]
def extendMask(image, x0, y0, x1, y1):
    a, b, xx = getab([[x0, y0], [x1, y1]], transpose = 1)
    print("a = %f, b = % f, xx = %f" % (a,b,xx))
    if xx == -1.:
        print('line not vertucal')
        if np.absolute(a) >= 1.:
            print('a >= 1')
#             print('looping from x = %d to %d' % (x0, x1))
#             if x0 > x1:
#                 inc = -1
#             else:
#                 inc = 1
#             for i in range(x0, x1, inc):
            for i in range(0, im_w):
                for j in range(math.ceil(np.absolute(a))): # to avoid line breaks when a > 1
                    if(int(a * i + b) + j > 0 and int(a * i + b) + j < im_w):
                        image[int(a * i + b) + j][i] = 1.
#                         print('hello')
        if np.absolute(a) < 1.:
            print('a < 1')
            for i in range(0, im_w):
                for j in range(math.ceil(np.absolute(1/a))): # to avoid line breaks when 1/a > 1
                    if(int(a * i + b) > 0 and int(a * i + b) < im_w):
                        image[i][int(a * i + b)] = 1.
                        print('hello2')
                        # b_n = int(a * i + b) + j - int(a_n * i)
    else:
        for i in range(im_w):
            image[i][xx] = 1

%matplotlib widget
base_mask = np.zeros((im_w, im_h)).astype(np.bool_)
extendMask(base_mask, df["x0"][0], df["y0"][0], df["x1"][0], df["y1"][0])
struct2 = ndimage.generate_binary_structure(2, 2)
base_mask = ndimage.binary_dilation(base_mask, structure=struct2, iterations = 10).astype(base_mask.dtype)
plt.imshow(base_mask)
np.count_nonzero(base_mask)

a = 1.963039, b = -1928.667351, xx = -1.000000
line not vertucal
a >= 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

298789

In [10]:
correlations = np.zeros(df.shape[0])
df.at[0, "group"] = 0
for i in range(0, len(correlations)):
    correlations[i] = np.count_nonzero(base_mask * mask[i+1])
    if correlations[i] > 0:
        df.at[i, "group"]= 0
#     print(i, correlations[i] / df["length"][i])
print(np.count_nonzero(correlations))

240


In [11]:
sdf = df[df['group'] == 0]
min_y = np.min([sdf['y0'].min(), sdf['y1'].min()])
max_y = np.max([sdf['y0'].max(), sdf['y1'].max()])
min_x = np.min([sdf['x0'].min(), sdf['x1'].min()])
max_x = np.max([sdf['x0'].max(), sdf['x1'].max()])

print(max_x)
print(min_x)
# min_x | min_y
# max_x | min_y
# min_x | max_y
# max_x | max_y

3365
1067


In [12]:
%matplotlib widget
newmask = np.zeros((full_image.shape[0],full_image.shape[1]), dtype = np.bool_)
drawLine(newmask, min_y, min_x, max_y, max_x)
struct2 = ndimage.generate_binary_structure(2, 2)
newmask = ndimage.binary_dilation(newmask, structure=struct2, iterations = 5).astype(newmask.dtype)
plt.imshow(newmask)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
df['length'][0]

711.101258612302

In [ ]:
%matplotlib widget
plt.imshow(cutoff_smaps[1][1])
plt.colorbar()
plt.show()

In [ ]:
full_smap_file = fits.open("/lustre/fs23/group/icecube/azenin/last/images/snr2_smap.fit")
full_smap = full_smap_file[0].data
image = full_smap
histo = image.flatten()
try:
    histo=histo[histogram < 0]
    quantile = np.quantile(histo, 0.32)
except:
    quantile = 0
cutoff = abs(quantile)
image[image < cutoff] = 0
plt.imshow(image)
plt.colorbar()
plt.show()


In [58]:
### efficiency metric
# make an expectation array with known streak parameters
# streak in images (x = ay + b):
a = 0
b = 1000
im_w = full_image.shape[0]
im_h = full_image.shape[1]
expectation_mask = np.zeros((im_w, im_h))
inv_expectation_mask = np.ones((im_w, im_h))
inv_mask = np.ones((im_w, im_h))
width = psf * 3

def widthfun(x):
    if (abs(x) <= width):
        return 1.
    else:
        return 0.

def AddMask(image, a = 0.5, b = 0, start = 0, end = -1, width = 2):
    if end == -1:
        end = im_w;
    if np.absolute(a) >= 1:
        for i in range(start, end):
            for j in range(math.ceil(np.absolute(a))): # to avoid line breaks when a > 1
                if(int(a * i + b) + j > 0 and int(a * i + b) + j < im_h):
                    image[i][int(a * i + b) + j] = 1.
            for k in range(math.ceil(np.absolute(a)), 3*width):
                if(int(a * i + b) + k > 0 and int(a * i + b) + k < im_h):
                    image[i][int(a * i + b) + k] += widthfun(k * np.cos(np.arctan(a)))

            for k in range(-3*int(np.ceil(width/2)), 0):
                if(int(a * i + b) + k  > 0 and int(a * i + b) + k < self.im_h):
                    image[i][int(a * i + b) + k] += widthfun(k * np.cos(np.arctan(a)))

    if np.absolute(a) < 1:
        for i in range(start, end):
            # for j in range(math.ceil(np.absolute(1/a))): # to avoid line breaks when 1/a > 1
            if(int(a * i + b) > 0 and int(a * i + b) < im_h):
                image[i][int(a * i + b)] += 1.
                    # b_n = int(a * i + b) + j - int(a_n * i)

            for k in range(1, 3*int(np.ceil(width/2))):
                if(int(a * i + b) + k > 0 and int(a * i + b) + k < im_h):
                    image[i][int(a * i + b) + k] += widthfun(k * np.cos(np.arctan(a)))
            for k in range(-3*int(np.ceil(width/2)), 0):
                if(int(a * i + b) + k  > 0 and int(a * i + b) + k < im_h):
                    image[i][int(a * i + b) + k] += widthfun(k * np.cos(np.arctan(a)))
                    
AddMask(expectation_mask, a, b, width = width)
expectation_mask = expectation_mask.astype(np.bool_)
inv_expectation_mask -= expectation_mask
inv_mask -= mask[0]
# compare with the result
targetpos = np.count_nonzero(expectation_mask > 0.1)
targetneg = np.count_nonzero(inv_expectation_mask > 0.1)
truepos = np.sum(mask[0][expectation_mask > 0.1])
falsepos = np.sum(mask[0][expectation_mask < 0.1])
efficiency_tp = truepos / targetpos
efficiency_fp = falsepos / targetneg 
print(efficiency_tp)
print(efficiency_fp)

0.5026919149726168
6.797266346849525e-06


In [ ]:
%matplotlib widget
plt.imshow(expectation_mask)
plt.colorbar()
plt.show()

In [25]:
# plt.imshow(mask)
%matplotlib widget
# plt.imshow(mask[0].astype(float))
plt.plot(mask[0][1000, :])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
df["x0"][2]

In [51]:
print(im_h)

6354
